Helper function for generating run_dir_list.ipynb which contains a list of subfolder names that contain experiment parameters to run

No need to run this if you already have the relevant run_dir_list.npy and/or cylinder_params_df/pkl files provided, but you can use this to generate arbitrary experiment parameter lists.

---
Generate run_dir_list.npy

Edit existing run_dir_list.npy

Generate cylinder_params_df.pkl

1. Generate run_dir_list

In [1]:
import numpy as np
import pandas as pd
def ctheta_to_alpha(cthetadeg): #ctheta is sector angle. alpha is half opening cone angle
    # alpha=theta/2=arcsin(phi/(2pi)), theta=full cone angle, alpha=half cone angle, phi=sector angle
    cthetarad=cthetadeg*np.pi/180
    alpharad=np.arcsin(cthetarad/(2*np.pi))
    alphadeg=alpharad*180/np.pi
    return alphadeg
def alpha_to_ctheta(alphadeg):
    alpharad=alphadeg*np.pi/180
    cthetarad=np.sin(alpharad)*(2*np.pi)
    cthetadeg=cthetarad*180/np.pi
    return cthetadeg

In [29]:
def magic_cinit_list(cinit_min=5,cinit_max=12):
#     #cinit range
#     cinit_min=5
#     cinit_max=12

    #for magic cylinders
    magic_cinit_list=np.arange(0,cinit_max+10,np.sqrt(3))
    magic_cinit_list=np.append(magic_cinit_list,np.arange(0,cinit_max+10,1))
    magic_cinit_list=np.unique(magic_cinit_list)

    #for magic cones - note that yinit changes with cone angle (i.e. low cone angles will initialize at very large yinit for a fixed circumference)
    magic_yinit_list=np.arange(0,cinit_max+10,np.sqrt(3)/2)
    magic_yinit_list=np.append(magic_yinit_list,np.arange(0,cinit_max+10,1))
    magic_yinit_list=np.unique(magic_yinit_list)

    #convert yinit_list to cinit_list for cthetadeg=60, since that's the magic cone angle that yinit concerns, then append to list.
    magic_cinit_list=np.append(magic_cinit_list,magic_yinit_list*(60/180*np.pi))

    #add additional cinit values to increase resolution
    magic_cinit_list=np.append(magic_cinit_list,np.arange(cinit_min,cinit_max,0.5))
    
    #filter based on cinit range
    magic_cinit_list=magic_cinit_list[np.where((magic_cinit_list<=cinit_max)&(magic_cinit_list>=cinit_min))]
    magic_cinit_list=np.unique(magic_cinit_list)

    if 5*np.sqrt(3) in magic_cinit_list: #ithetadeg=60 magic cylinder case
        print('#ithetadeg=60 magic cylinder case included')
    if 5 in magic_cinit_list: #ithetadeg=0 magic cylinder case
        print('#ithetadeg=0 magic cylinder case included')
    if 7*np.sqrt(3)/2*(60/180*np.pi) in magic_cinit_list: #ithetadeg=0 magic cone case
        print('#ithetadeg=0 magic cone case included')
    if 7*(60/180*np.pi) in magic_cinit_list: #ithetadeg=60 magic cone case
        print('#ithetadeg=60 magic cone case included')
    return magic_cinit_list

In [30]:
magic_cinit_list(cinit_min=0,cinit_max=4.99)

array([0.        , 0.5       , 0.90689968, 1.        , 1.04719755,
       1.5       , 1.73205081, 1.81379936, 2.        , 2.0943951 ,
       2.5       , 2.72069905, 3.        , 3.14159265, 3.46410162,
       3.5       , 3.62759873, 4.        , 4.1887902 , 4.5       ,
       4.53449841])

In [42]:
def generate_run_dir_list():
    # cthetadeg_list=np.linspace(0,60,11)
    # cinit_list=np.linspace(5,12,8)
    # ithetadeg_list=np.linspace(0,30,4)
    # trials_list=np.arange(0,50,1)
    # Nmax_list=np.linspace(1000,1000,1)
    cthetadeg_list=np.linspace(0,30,7) #cylinders and cones
    # cthetadeg_list=[0] #cylinders only
    # cinit_list=magic_cinit_list(cinit_min=0,cinit_max=12)
    cinit_list=np.linspace(0,12,49)
#     cinit_list=np.append(cinit_list,np.arange(5,12,np.sqrt(3)/2))

#     ## magic cone needs to calculate cinit from integer values of __yinit__ (sectorRinit)!
#     cthetadeg=60
#     yinit_list=np.linspace(5,12,15)
#     cinit_list=yinit_list*(cthetadeg*np.pi/180) #yinit=sectorRinit; sectorRinit*cthetarad=cinit

    ithetadeg_list=np.linspace(59,60,1)
    trials_list=np.arange(0,100,1)
    Nmax_list=np.linspace(1000,1000,1)

    print('\ncthetadeg \n',cthetadeg_list)
    # print('thetadeg',ctheta_to_alpha(cthetadeg_list*np.pi/180)*2*180/np.pi)
    print('cinit \n',cinit_list)
    print('ithetadeg \n',ithetadeg_list)
    print('trials \n',trials_list)
    print('Nmax \n',Nmax_list)

    run_dir_list=[]
    for trial in trials_list:
        for cthetadeg in cthetadeg_list:
            for cinit in cinit_list: #circumference to initialize seed at, equivalent to arc length. cinit=s=r*ctheta=yinit*ctheta
                for ithetadeg in ithetadeg_list:
                    for Nmax in Nmax_list: #won't be max packing for cylinders but should be sufficient >1deg
                        run_dir = ['%s-%s_%s-%s_%s-%s_%s-%s_%s-%s' % \
                                  ('cthetadeg', str(cthetadeg), 'cinit', str(cinit),'ithetadeg', str(ithetadeg),'Nmax', str(Nmax),'trial', str(trial))] #_%s, str(rep)
                        run_dir_list+=run_dir
    run_dir_list=np.array(run_dir_list)
    filename='run_dir_list_022723_'+str(np.min(trials_list))+'-'+str(np.max(trials_list))#+'_cylinders0-5cinit'
    np.save(filename,run_dir_list)
    print('runtime: ~'+str(int(((len(run_dir_list)*0.5)/60)/24))+' days')
    print(filename)
    return run_dir_list
run_dir_list=generate_run_dir_list()


cthetadeg 
 [ 0.  5. 10. 15. 20. 25. 30.]
cinit 
 [ 0.    0.25  0.5   0.75  1.    1.25  1.5   1.75  2.    2.25  2.5   2.75
  3.    3.25  3.5   3.75  4.    4.25  4.5   4.75  5.    5.25  5.5   5.75
  6.    6.25  6.5   6.75  7.    7.25  7.5   7.75  8.    8.25  8.5   8.75
  9.    9.25  9.5   9.75 10.   10.25 10.5  10.75 11.   11.25 11.5  11.75
 12.  ]
ithetadeg 
 [59.]
trials 
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Nmax 
 [1000.]
runtime: ~11 days
run_dir_list_022723_0-99


In [43]:
len(run_dir_list)

34300

In [44]:
run_dir_list[1500:]

array(['cthetadeg-10.0_cinit-7.5_ithetadeg-59.0_Nmax-1000.0_trial-4',
       'cthetadeg-10.0_cinit-7.75_ithetadeg-59.0_Nmax-1000.0_trial-4',
       'cthetadeg-10.0_cinit-8.0_ithetadeg-59.0_Nmax-1000.0_trial-4', ...,
       'cthetadeg-30.0_cinit-11.5_ithetadeg-59.0_Nmax-1000.0_trial-99',
       'cthetadeg-30.0_cinit-11.75_ithetadeg-59.0_Nmax-1000.0_trial-99',
       'cthetadeg-30.0_cinit-12.0_ithetadeg-59.0_Nmax-1000.0_trial-99'],
      dtype='<U62')

In [47]:
len(run_dir_list)/5000

6.86

In [46]:
len(run_dir_list)/492

69.71544715447155

In [28]:
#list # of jobs if wish to run in batches i.e. on cluster (this will need to be manually specified in simulation file or system input
job_list=''
for item in np.arange(1,51,1).tolist():
    job_list+=str(item)+' '
job_list

'1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 '

In [29]:
'cthetadeg-5.0_cinit-5.0_ithetadeg-0.0_Nmax-1000.0_trial-14'.split('_')[1]

'cinit-5.0'

In [35]:
def edit_run_dir_list():
    edit_existing=np.load('/Volumes/manoharan_lab/jessicasun/Simulations/022322 bennett simulation/run_dir_list_022322_0-99.npy')
    
    #13-25,37-61,74-86 remaining
    #last time it took 4 jobs, running 7 days, to do around 50 trials.. (12 trials per job)
#     remaining_trials=np.arange(13,25+1,1) #include edge trials, JUST in case
#     remaining_trials=np.append(remaining_trials,np.arange(37,61+1,1))
#     remaining_trials=np.append(remaining_trials,np.arange(74,86+1,1))

    remaining_trials=np.array([14])
    remaining_cthetadeg=np.array([5.0])
    remaining_cinit=np.array([5.0])
    remaining_ithetadeg=np.array([10.0])
    
    run_dir_list_remaining=[]
    for i in edit_existing:
        trial_num=int(i.split('-')[-1])
        cthetadeg=float(i.split('_')[0].split('-')[1])
        cinit=float(i.split('_')[1].split('-')[1])
        ithetadeg=float(i.split('_')[2].split('-')[1])        
        if trial_num in remaining_trials:
            if cthetadeg in remaining_cthetadeg:
                if cinit in remaining_cinit:
                    if ithetadeg in remaining_ithetadeg:
                        run_dir_list_remaining+=[i]    
    np.save('run_dir_list_remaining',run_dir_list_remaining)            
    return run_dir_list_remaining
run_dir_list_remaining=edit_run_dir_list()

In [36]:
run_dir_list_remaining

['cthetadeg-5.0_cinit-5.0_ithetadeg-10.0_Nmax-1000.0_trial-14']

In [33]:
len(run_dir_list_remaining)/8

8701.875

In [28]:
len(run_dir_list_remaining)

69615

In [10]:
len(run_dir_list_remaining)

13650

In [8]:
# 2023 followup to get general commensurate cylinder circumference and orientation

def get_cylinder_cinit(m,n,a): #find commensurate cylinder cinit for a given initial orientation
    cinit=a*np.sqrt(m**2+n**2-(m*n))
    return cinit
def get_cylinder_ithetadeg(m,n):
    mn_ratio=m/n
    phi_dbeller=np.arctan((2/np.sqrt(3))*((mn_ratio)-(1/2))) #phi based on DA Beller, DR Nelson convention (angle made with vertical axis)
    phi_dbeller_deg=phi_dbeller*180/np.pi
    ithetadeg=(90-phi_dbeller_deg) #theta based on our convention, angle made with circumference
    return phi_dbeller_deg,ithetadeg
def get_cylinder_params(m,n,a):
    phi_dbeller_deg,ithetadeg=get_cylinder_ithetadeg(m,n)
    cinit=get_cylinder_cinit(m,n,a)
    mn_ratio=m/n
    cylinder_params_df=pd.DataFrame([{'m':m,'n':n,'mn_ratio':mn_ratio,'phi_dbeller_deg':phi_dbeller_deg,'ithetadeg':ithetadeg,'a':a,'cinit':cinit}])
    return cylinder_params_df

In [9]:
a_list=np.arange(1,11,1)
cylinder_params_df_list=[]
mn_list=[(1,1),(2,1),(3,2),(4,3),(5,4)]
for mn in mn_list:
    m,n=mn[0],mn[1]
    for a in a_list:
        cylinder_params_df=get_cylinder_params(m,n,a)
        cylinder_params_df_list+=[cylinder_params_df]    
cylinder_params_df=pd.concat(cylinder_params_df_list)
cylinder_params_df=cylinder_params_df[cylinder_params_df['cinit']<=12] #from experience i know that simulations break down at high circumferences due to numerical error

In [10]:
cylinder_params_df

,m,n,mn_ratio,phi_dbeller_deg,ithetadeg,a,cinit
0,1,1,1.000000,30.000000,60.000000,1,1.000000
0,1,1,1.000000,30.000000,60.000000,2,2.000000
0,1,1,1.000000,30.000000,60.000000,3,3.000000
0,1,1,1.000000,30.000000,60.000000,4,4.000000
0,1,1,1.000000,30.000000,60.000000,5,5.000000
0,1,1,1.000000,30.000000,60.000000,6,6.000000
0,1,1,1.000000,30.000000,60.000000,7,7.000000
0,1,1,1.000000,30.000000,60.000000,8,8.000000
0,1,1,1.000000,30.000000,60.000000,9,9.000000
0,1,1,1.000000,30.000000,60.000000,10,10.000000


In [129]:
cylinder_params_df.to_pickle('cylinder_params_df.pkl')

In [124]:
def generate_cylinder_run_dir_list(cylinder_params_df):
    cthetadeg_list=[0] #cylinders only
    cinit_list=np.unique(np.array(cylinder_params_df['cinit']))

    ithetadeg_list=np.unique(np.array(cylinder_params_df['ithetadeg']))
    trials_list=np.arange(0,3,1)
    Nmax_list=np.linspace(1000,1000,1)

    print('\ncthetadeg \n',cthetadeg_list)
    print('cinit \n',cinit_list)
    print('ithetadeg \n',ithetadeg_list)
    print('trials \n',trials_list)
    print('Nmax \n',Nmax_list)

    run_dir_list=[]
    for trial in trials_list:
        for cthetadeg in cthetadeg_list:
            for cinit in cinit_list: #circumference to initialize seed at, equivalent to arc length. cinit=s=r*ctheta=yinit*ctheta
                for ithetadeg in ithetadeg_list:
                    for Nmax in Nmax_list: #won't be max packing for cylinders but should be sufficient >1deg
                        run_dir = ['%s-%s_%s-%s_%s-%s_%s-%s_%s-%s' % \
                                  ('cthetadeg', str(cthetadeg), 'cinit', str(cinit),'ithetadeg', str(ithetadeg),'Nmax', str(Nmax),'trial', str(trial))] #_%s, str(rep)
                        run_dir_list+=run_dir
    run_dir_list=np.array(run_dir_list)
    filename='run_dir_list_022323_'+str(np.min(trials_list))+'-'+str(np.max(trials_list))+'cinit0-12'#+'_cylinders0-5cinit'
    np.save(filename,run_dir_list)
    print('runtime: ~'+str(int(((len(run_dir_list)*0.5)/60)/24))+' days')
    print(filename)
    return run_dir_list
run_dir_list=generate_cylinder_run_dir_list(cylinder_params_df)


cthetadeg 
 [0]
cinit 
 [ 1.          1.73205081  2.          2.64575131  3.          3.46410162
  3.60555128  4.          4.58257569  5.          5.19615242  5.29150262
  6.          6.92820323  7.          7.21110255  7.93725393  8.
  8.66025404  9.          9.16515139 10.         10.39230485 10.58300524
 10.81665383]
ithetadeg 
 [30.         40.89339465 46.10211375 49.10660535 60.        ]
trials 
 [0 1 2]
Nmax 
 [1000.]
runtime: ~0 days
run_dir_list_022323_0-2cinit0-12


In [125]:
len(run_dir_list)/492

0.7621951219512195